# PDIoT Group A1 model

### Imports

In [1]:
import os
import numpy as np
import tensorflow as tf

from ml.config import cfg
from ml.dataset import odgt2data
from ml.models import ModelBuilder, OptimizerBuilder, LRScheduleBuilder

print("All packages imported!")

2023-11-09 16:14:41.709810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


All packages imported!


### Config

In [2]:
config_fp = '/home/jonathan/git/pdiot-ml/cfg/log-motion.yaml'
cfg.merge_from_file(config_fp)
cfg.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg.TRAIN.path = '/home/jonathan/git/pdiot-ml/ckpt/log-motion/'    

### Data

In [3]:
train_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.val)

train_X, train_y = odgt2data(train_odgt_fp, cfg.MODEL.INPUT.window_size, 
                                            cfg.TRAIN.DATA.overlap_size, 
                                            cfg.DATASET.num_classes)
val_X, val_y = odgt2data(val_odgt_fp, cfg.MODEL.INPUT.window_size, 
                                      cfg.TRAIN.DATA.overlap_size,
                                      cfg.DATASET.num_classes)

### Model

In [4]:
model = ModelBuilder.build_classifier(cfg.MODEL, '', cfg.DATASET.num_classes)
optimizer = OptimizerBuilder.build_optimizer(cfg.TRAIN.OPTIM)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
    tf.keras.metrics.Precision(name='prec'),
    tf.keras.metrics.Recall(name='rec'),
    tf.keras.metrics.AUC(name='auc')
]


lr_scheduler = LRScheduleBuilder.build_scheduler(cfg.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg.TRAIN.LEN.early_stop)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(cfg.TRAIN.path, 'weights.hdf5'),
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

history_callback = tf.keras.callbacks.CSVLogger(
    os.path.join(cfg.TRAIN.path, 'history.csv'), 
    separator=',', 
    append=False)


model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_X, train_y, validation_data=(val_X, val_y), 
          epochs=cfg.TRAIN.LEN.num_epoch, 
          batch_size=cfg.TRAIN.DATA.batch_size,
          callbacks=[lr_callback, 
                     early_stop_callback,
                     checkpoint_callback,
                     history_callback])

Epoch 1/100


2023-11-09 16:14:47.139662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 16:14:47.141262: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


89/89 [==============================] - 1s 11ms/step - loss: 1.7885 - acc: 0.6392 - prec: 0.6392 - rec: 0.6392 - auc: 0.6554 - val_loss: 1.4160 - val_acc: 0.7086 - val_prec: 0.7086 - val_rec: 0.7086 - val_auc: 0.7233 - lr: 0.0010
Epoch 2/100
89/89 [==============================] - 1s 8ms/step - loss: 0.9255 - acc: 0.7595 - prec: 0.7595 - rec: 0.7595 - auc: 0.7758 - val_loss: 0.9186 - val_acc: 0.7748 - val_prec: 0.7748 - val_rec: 0.7748 - val_auc: 0.7823 - lr: 0.0010
Epoch 3/100
89/89 [==============================] - 1s 7ms/step - loss: 0.8326 - acc: 0.8133 - prec: 0.8133 - rec: 0.8133 - auc: 0.8061 - val_loss: 0.6667 - val_acc: 0.8278 - val_prec: 0.8278 - val_rec: 0.8278 - val_auc: 0.8154 - lr: 0.0010
Epoch 4/100
89/89 [==============================] - 1s 8ms/step - loss: 0.8960 - acc: 0.8230 - prec: 0.8230 - rec: 0.8230 - auc: 0.8146 - val_loss: 0.7374 - val_acc: 0.8675 - val_prec: 0.8675 - val_rec: 0.8675 - val_auc: 0.8517 - lr: 0.0010
Epoch 5/100
89/89 [========================